# Keras + tensorflow serving

* Convert the keras model to a tf model and save it
* Deploy the model with tensorflow-serving
* Get sequences from the tensorflow-serving


## Convert the keras model to a tf model and save it

Based in [keras-tf](https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html)




In [1]:
import sys
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text
from keras.layers import Dense, Activation, GRU
from keras.models import Sequential
from keras.layers import InputLayer
import os

Using TensorFlow backend.


In [3]:
# Defining the model
def create_gru_model( num_chars):
    """
    Define the network
    :param
        numbers_chars .- Number chars using in the training process
    :return:
        model .- Model network defined
    """
    model = Sequential()
    # 1 Layer .- GRU layer 1 should be an GRU module with 200 hidden units
    model.add(GRU(200, input_shape=(window_size, num_chars), return_sequences=True))
    # 2 Layer .- GRU layer 2 should be an GRU module with 200 hidden units
    model.add(GRU(200))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(Dense(num_chars, activation='softmax'))
    return model

**Important: the model is going to be used as predict, then disable training phase**

In [ ]:
import keras.backend as K
K.set_learning_phase(0)

In [4]:
# Input size of the network, the entry text must have the same length
window_size = 100
# Get dictionaries
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# regenerate the model
model=create_gru_model(number_chars)
model.load_weights('../model_weights/best_beiras_gru_textdata_weights.hdf5')


Save the model as tensorflow model

In [6]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

# Path to export, 1 is the version, 
# we can serve differents version with the same server
export_path = "../export-tf/1"
import shutil
import os


if os.path.isdir(export_path):
    shutil.rmtree(export_path)
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'sequence': model.input},
                                  outputs={'scores': model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../export-tf/1/saved_model.pb'


In [7]:
#Input and output shape
print(model.input)
print(model.output)

Tensor("gru_1_input:0", shape=(?, 100, 55), dtype=float32)


The PB file describes the model. 
If we use builder.save(astext=True), the PB is in text mode.

## Deploy the model with tensorflow-serving

### Install 

```sh 
echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list
```

```sh 
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -
```


```sh 
sudo apt-get update && sudo apt-get install tensorflow-model-server
```

**The tensorflow-model-server version do not soport GPU**

### Lanch server

tensorflow_model_server --port=9000 --model_name=default --model_base_path=/home/aind2/beiras-rnn/export-tf


## Get sequences from the tensorflow-serving

Based in [How to deploy Machine Learning models ](https://medium.com/towards-data-science/how-to-deploy-machine-learning-models-with-tensorflow-part-2-containerize-it-db0ad7ca35a7)

**In pip repository, there is  tensorflow-serving  API for python2. For python3 we must generate it.**

```sh
git clone --recurse-submodules https://github.com/tensorflow/serving.git

cd <tensorflow serving source folder>
# 2
mv ./tensorflow ./tensorflow_
mv ./tensorflow_/tensorflow .
# 3
python -m grpc.tools.protoc ./tensorflow_serving/apis/*.proto --python_out=<path to GAN project> --grpc_python_out=<path to GAN project> --proto_path=.
# 4
mv ./tensorflow ./tensorflow_
mv ./tensorflow_ ./tensorflow
```

In [16]:
import sys
import numpy as np
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"
# Load values
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# Clean the text
input_clean=clean_text(input_init.lower())
input_clean = input_clean[:window_size]
# Text to array [1,input_lenght,num_chars]
x_test = np.zeros((1,window_size, number_chars))
for t, char in enumerate(input_clean):
    x_test[0, t, chars_to_indices[char]] = 1.
x_test    
    


In [18]:
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import tensorflow as tf
import grpc

# Get the array with the probabilities for the next charazter
channel = grpc.insecure_channel("localhost:" + str(9000))
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest()
# Name of the model
request.model_spec.name = 'default' 
request.model_spec.signature_name = 'predict' 

request.inputs['inputs'].CopyFrom( 
        tf.contrib.util.make_tensor_proto(
            x_test,dtype='float32'))
result=stub.Predict(request)


In [19]:
# Get the charazter from array
test_predict=np.array(result.outputs['outputs'].float_val)
r = np.argmax(test_predict)  # predict class of each test input
d = indices_to_chars[r]
d

' '

In [24]:
# Define a function
def predict_one(text_predict,stub,window_size,number_chars):
    # Convert input sequence to array
    x_test = np.zeros((1,window_size, number_chars))
    for t, char in enumerate(text_predict):
        x_test[0, t, chars_to_indices[char]] = 1.
    #Prepare the request
    request = predict_pb2.PredictRequest() 
    request.model_spec.name = 'default' 
    request.model_spec.signature_name = 'predict' 
    request.inputs['inputs'].CopyFrom( 
        tf.contrib.util.make_tensor_proto(
            x_test,dtype='float32'))
    #Made the request
    result=stub.Predict(request)
    # Convert the request return to a charazter
    test_predict=np.array(result.outputs['outputs'].float_val)
    r = np.argmax(test_predict)  # predict class of each test input
    return (indices_to_chars[r])

In [9]:
# Use the function
input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
input_clean=clean_text(input_init.lower())
channel = grpc.insecure_channel("localhost:" + str(9000))
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
d=predict_one(input_clean,stub,window_size,number_chars)
print(d)

se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu
m


In [10]:
# Complete a sequence using the server
def predict_window(text_predict,number_predict,window_size):
    # Get dictionaries
    chars_to_indices, indices_to_chars = load_coded_dictionaries()
    number_chars=len(chars_to_indices)
    # Clean the test
    input_clean=clean_text(text_predict.lower())
    # Get stub
    channel = grpc.insecure_channel("localhost:" + str(9000))
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    # Call server for all charazters
    for i in range(number_predict):
        d=predict_one(input_clean[i:],stub,window_size,number_chars)
        input_clean+=d
    return input_clean
        

In [27]:
#Test predict window
beiras_said="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"
#beiras_said="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
text=predict_window(beiras_said,window_size,window_size)
text

'pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran un contrasentido arestora e a construción de anos de autonomía galega non é unha concepción do seu '